# Xe atomic data: tests for Pandas and web scraping
08/06/24

Current results per Table 1 in the manuscript, copied and reformatted to `Quantum-Beat_Photoelectron-Imaging_Spectroscopy_of_Xe_in_the_VUV/scratch/Xe_data_table.ods`

TODO: check old docs, may have .ods stuff there too (but not on OSF).

UPDATE: yes, see `Quantum-Beat_Photoelectron-Imaging_Spectroscopy_of_Xe_in_the_VUV/scratch/Xe_hyperfine_FT_features_270717.xls` Note this has values for different FT runs/windows etc. too.

TODO: uncertainties handling...?

UPDATE: yes, seems to be working OK with uncertainties after some string fixes.

## Test ODS/CSV read

May need `pip install odfpy` for Open Doc files

In [69]:
import pandas as pd

filePath='../../scratch/Xe_data_table_fixedFractions.ods'
rawXeHyperfineResults = pd.read_excel(filePath, sheet_name=1)
rawXeHyperfineResults

,Isotope,I,A/MHz,B/MHz,F,F′,Splitting/cm−1
0,129,0.5,-5723 (9),-,0.5,1.5,0.2863 (5)
1,131,1.5,1697 (30),-8 (7),1.5,0.5,0.0855 (10)
2,131,1.5,1697 (30),-8 (7),2.5,1.5,0.1411 (29)
3,131,1.5,1697 (30),-8 (7),2.5,0.5,0.2276 (29)


In [12]:
# Tidy up...
# Note - may also need to force dtypes here...?
rawXeHyperfineResults.set_index(['Isotope','I','F'])

A/MHz   B/MHz   F′ Splitting/cm−1
Isotope I   F                                         
129     ½   ½    -5723 (9)       -  3/2     0.2863 (5)
131     3/2 3/2  1697 (30)  -8 (7)    ½    0.0855 (10)
            5/2  1697 (30)  -8 (7)  3/2    0.1411 (29)
            5/2  1697 (30)  -8 (7)    ½    0.2276 (29)

In [13]:
rawXeHyperfineResults.dtypes

Isotope            int64
I                 object
A/MHz             object
B/MHz             object
F                 object
F′                object
Splitting/cm−1    object
dtype: object

In [3]:
!which pip

/home/paul/anaconda3/envs/epsdev-shared-100122/bin/pip


In [4]:
!pip install odfpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 717.0/717.0 kB 8.5 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for odfpy: filename=odfpy-1.4.1-py2.py3-none-any.whl size=160691 sha256=658b740390f8095fe2aab2296a7e403513c428a084a0ba296fc25ed872a13106
  Stored in directory: /home/paul/.cache/pip/wheels/e2/f4/5d/a68c656235d33455a1d0f78e877acddfa006907a6d52d7e6ee
Successfully built odfpy


## Test conversion with uncertainties

In [4]:
rawXeHyperfineResults.columns

Index(['Isotope', 'I', 'A/MHz', 'B/MHz', 'F', 'F′', 'Splitting/cm−1'], dtype='object')

In [9]:
rawXeHyperfineResults['A/MHz'].str.replace(' ','')   #.apply(string.strip)

0    -5723(9)
1    1697(30)
2    1697(30)
3    1697(30)
Name: A/MHz, dtype: object

In [12]:
rawXeHyperfineResults.replace(' ','')  # Index only?

,Isotope,I,A/MHz,B/MHz,F,F′,Splitting/cm−1
0,129,½,-5723 (9),-,½,3/2,0.2863 (5)
1,131,3/2,1697 (30),-8 (7),3/2,½,0.0855 (10)
2,131,3/2,1697 (30),-8 (7),5/2,3/2,0.1411 (29)
3,131,3/2,1697 (30),-8 (7),5/2,½,0.2276 (29)


In [63]:
# Lambda map...
# Works for sub-selected cols
# rawXeHyperfineResults[['A/MHz', 'B/MHz']].apply(lambda x: x.str.replace(' ',''))

# Fails for full DF?
# rawXeHyperfineResults.apply(lambda x: x.str.replace(' ','') if isinstance(x, str) else x, axis=1)

# Applymap works overall - works elementwise.
tidied = rawXeHyperfineResults.applymap(lambda x: x.replace(' ','') if isinstance(x, str) else x)
tidied = tidied.replace('-','nan(nan)')
tidied

,Isotope,I,A/MHz,B/MHz,F,F′,Splitting/cm−1
0,129,0.5,-5723(9),nan(nan),0.5,1.5,0.2863(5)
1,131,1.5,1697(30),-8(7),1.5,0.5,0.0855(10)
2,131,1.5,1697(30),-8(7),2.5,1.5,0.1411(29)
3,131,1.5,1697(30),-8(7),2.5,0.5,0.2276(29)


In [64]:
from uncertainties import ufloat_fromstr

# tidied['A/MHz'].apply(lambda x: ufloat_fromstr(x))  # OK

uList = ['A/MHz','B/MHz','Splitting/cm−1']

tidied[uList] = tidied[uList].applymap(lambda x: ufloat_fromstr(x))  # OK

In [52]:
tidied['A/MHz'][0]

-5723.0+/-9.0

In [65]:
# Fix fractions... EASIER TO DO THIS WITH SYMPY, or in source data...?
# UPDATE: now fixed, set "fraction" rather than "text" in ODS sheet cell formats.

# tidied[['I','F','F′']].astype(sympy)
# tidied[['I','F','F′']].to_numeric()

fList = ['I','F','F′']

# tidied[fList].applymap(lambda x: x.split("/"))  #.apply(lambda x: float(x[0]) / float(x[1]))
tidied[fList]

,I,F,F′
0,0.5,0.5,1.5
1,1.5,1.5,0.5
2,1.5,2.5,1.5
3,1.5,2.5,0.5


In [66]:
tidied.dtypes

Isotope             int64
I                 float64
A/MHz              object
B/MHz              object
F                 float64
F′                float64
Splitting/cm−1     object
dtype: object

In [67]:
tidied.set_index(['Isotope','I','F'], inplace=True)

In [68]:
tidied

A/MHz      B/MHz   F′   Splitting/cm−1
Isotope I   F                                              
129     0.5 0.5  -5723+/-9  nan+/-nan  1.5  0.2863+/-0.0005
131     1.5 1.5  1697+/-30     -8+/-7  0.5  0.0855+/-0.0010
            2.5  1697+/-30     -8+/-7  1.5  0.1411+/-0.0029
            2.5  1697+/-30     -8+/-7  0.5  0.2276+/-0.0029